In [136]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/trains/train.csv
/kaggle/input/trains/test.csv


In [137]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from category_encoders import CountEncoder
from sklearn.model_selection import train_test_split

In [138]:
import pandas as pd
test = pd.read_csv("../input/trains/test.csv")
train = pd.read_csv("../input/trains/train.csv")

In [139]:
dfCheto = pd.concat([train,test])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [140]:
dfCheto['precio'] = SimpleImputer(strategy='median').fit_transform(dfCheto[['precio']])

In [141]:
dfCheto.columns

Index(['antiguedad', 'banos', 'centroscomercialescercanos', 'ciudad',
       'descripcion', 'direccion', 'escuelascercanas', 'fecha', 'garages',
       'gimnasio', 'habitaciones', 'id', 'idzona', 'lat', 'lng',
       'metroscubiertos', 'metrostotales', 'piscina', 'precio', 'provincia',
       'tipodepropiedad', 'titulo', 'usosmultiples'],
      dtype='object')

In [192]:
df = dfCheto.copy()

In [193]:
df.drop(columns=['usosmultiples', 'piscina', 'lat', 'lng', 'gimnasio', 'escuelascercanas', 'direccion', 'centroscomercialescercanos'], inplace = True)

In [194]:
df.columns

Index(['antiguedad', 'banos', 'ciudad', 'descripcion', 'fecha', 'garages',
       'habitaciones', 'idzona', 'metroscubiertos', 'metrostotales', 'precio',
       'provincia', 'tipodepropiedad', 'titulo'],
      dtype='object')

In [195]:
df.titulo.fillna('', inplace=True)
df.descripcion.fillna('', inplace=True)
df['texto'] = df.titulo + ' ' + df.descripcion
df['texto'] = df.texto.str.len()
df.drop(columns = ['descripcion', 'titulo'], inplace = True)

In [196]:
df['ciudad'] = SimpleImputer(strategy='constant', fill_value='lomadelejos').fit_transform(df[['ciudad']])
df['ciudad'] = CountEncoder().fit_transform(df['ciudad'])

In [197]:
df['antiguedad'] = SimpleImputer(strategy='median').fit_transform(df[['antiguedad']])
df['habitaciones'] = SimpleImputer(strategy='median').fit_transform(df[['habitaciones']])

In [198]:
df.garages.fillna(0, inplace=True)

In [199]:
aux2 = dfCheto[['idzona', 'ciudad']].copy()
aux2.dropna(inplace = True)
aux2 = aux2.groupby('ciudad').agg({'idzona':'mean'})
ids = aux2.T.to_dict('records').copy()
ids = ids[0]

In [200]:
df.idzona.fillna(df.ciudad.map(ids), inplace = True)

In [201]:
df['idzona'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['idzona']])

In [202]:
df['tipodepropiedad'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['tipodepropiedad']])
aux = df[['banos','tipodepropiedad']].copy()
aux = aux.groupby('tipodepropiedad').agg({'banos':'median'})
banos = aux.T.to_dict('records').copy()
banos = banos[0]

In [203]:
df.banos.fillna(df.tipodepropiedad.map(banos), inplace = True)

In [204]:
df.banos.fillna(1, inplace=True)

In [205]:
df["swapear"] = df["metrostotales"] < df["metroscubiertos"]
df_metros_arreglados = df.copy()
aux = df_metros_arreglados.loc[df_metros_arreglados["swapear"], "metrostotales"]
if (not aux.empty):
    df_metros_arreglados.loc[df_metros_arreglados["swapear"],"metrostotales"] = \
    df_metros_arreglados.loc[df_metros_arreglados["swapear"], "metroscubiertos"]
    df_metros_arreglados.loc[df_metros_arreglados["swapear"], "metroscubiertos"] = aux
df_metros_arreglados.drop(columns="swapear", inplace=True)
(df_metros_arreglados["metrostotales"] < df_metros_arreglados["metroscubiertos"]).value_counts()
df = df_metros_arreglados

In [206]:
df['fecha'] = pd.to_datetime(df['fecha'])
df['dia'] = df.fecha.dt.day
df['mes'] = df.fecha.dt.month
df['anio'] = df.fecha.dt.year
df.drop(columns=['fecha'], inplace=True)

In [207]:
transformador_categorias = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
])

transfomador_numero = Pipeline(steps=[('a', SimpleImputer())])

In [208]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', transfomador_numero, ['ciudad', 'antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales', 'idzona', 'texto', 'dia', 'mes', 'anio']),
        ('cat', transformador_categorias, ['tipodepropiedad', 'provincia'])
    ])

In [209]:
import xgboost as xgb
model = xgb.XGBRegressor(learning_rate= 0.1,n_estimators= 1500,scale_pos_weight= 2,max_depth= 5,min_child_weight= 5,gamma= 0.0,colsample_bytree= 0.7500000000000001,
                     subsample= 0.7,colsample_bylevel= 0.65, tree_method='gpu_hist', gpu_id=0)

In [210]:
train.shape

(240000, 23)

In [211]:
test.shape

(60000, 22)

In [212]:
entrenamiento = df.head(240000).copy()

In [213]:
entrenamiento.shape

(240000, 15)

In [214]:
prueba = df.tail(60000)

In [215]:
prueba.drop(columns=['precio'], inplace=True)

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [219]:
train_x = entrenamiento.drop(columns=['precio']).copy()
train_y = entrenamiento.precio.copy()
#train_x, test_x, train_y, test_y = train_test_split(train_x, entrenamiento.precio,train_size=0.8, test_size=0.2, random_state = 0)

In [220]:
train_x.tipodepropiedad = train_x.tipodepropiedad.astype(str)
train_x.provincia = train_x.provincia.astype(str)
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

In [221]:
my_pipeline.fit(train_x, train_y)

/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[17:56:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('a',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='mean',
                                                                     

In [225]:
preds = my_pipeline.predict(prueba)

In [229]:
from sklearn.metrics import mean_absolute_error
#mean_absolute_error(preds, test_y)

In [227]:
res = pd.DataFrame(preds, index=prueba.id, columns=['precio'])
res.reset_index(inplace=True)
res.columns = ["id", "target"]
display(res.head())
# RMSLE=1.0249284784393988 ?


# import the modules we'll need
from IPython.display import HTML
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "knn-results.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe
create_download_link(res)

AttributeError: 'DataFrame' object has no attribute 'id'